# Kaggle ML - CBHY #


## Weather Datasets ##

In [ ]:
# import Lib for DeepLearning
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import folium
import os
import sys


# import CodeByHY as CBHY



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout, LSTM, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Transfer Learning Model in ImageNet
from tensorflow.keras.applications import VGG16, Xception


In [ ]:
from google.colab import drive

drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/mljnu/cnu-mlclass2/train.csv")

### Train DataFrame 분포 ###

In [ ]:
train_df.head()

,image,label
0,rain_418.jpg,rain
1,snow_150.jpg,snow
2,sunny_809.jpg,sunny
3,fog_750.jpg,fog
4,dust_646.jpg,dust


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4355 entries, 0 to 4354
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   4355 non-null   object
 1   label   4355 non-null   object
dtypes: object(2)
memory usage: 68.2+ KB


column 2개, 결측치 없음, Dtype 모두 object  
  
image = 사진 파일 이름  
label = 날씨(정답값)

In [ ]:
train_df.label.value_counts()

sunny    1071
rain      857
snow      857
dust      820
fog       750
Name: label, dtype: int64

In [ ]:

TRAINING_DIR = "/content/drive/MyDrive/mljnu/cnu-mlclass2/train"

training_datagen = ImageDataGenerator(
    rescale=1. / 255, # Normalization
    rotation_range=40, 
    width_shift_range=0.35,
    height_shift_range=0.35,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
        fill_mode='nearest', 
    validation_split=0.06,
    )






In [ ]:
# IDG 적용 
training_generator = training_datagen.flow_from_directory(TRAINING_DIR, # root 폴더 경로
                                                          batch_size=32, 
                                                          target_size=(224, 224), 
                                                          class_mode='categorical', # 다중 분류
                                                          subset='training',
                                                         )

validation_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                          batch_size=32, 
                                                          target_size=(224, 224), 
                                                          class_mode='categorical',
                                                          subset='validation', 
                                                         )

Found 4095 images belonging to 5 classes.
Found 260 images belonging to 5 classes.


1. flow_from_driectory 사용,  
BastLine Code에서 flow_from_dataframe 사용  
굳이 DataFrame으로 받은 이유를 잘 모르겠음
  
   
2. 굳이 Split단계에서 sklearn.model_selection.train_test_split 사용한 이유 잘 모르겠음

In [ ]:
transfer_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
transfer_model.trainable=True

model = Sequential([
    transfer_model,
    Flatten(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'), 
    Dense(256, activation='relu'), 
    Dense(128, activation='relu'), 
    Dense(64, activation='relu'), 
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'), 
    Dense(5, activation='softmax'),
])

83697664/83683744 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dropout (Dropout)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 256)               1

In [ ]:
checkpoint_path = "my_checkpoint1.ckpt" # 체크포인트 위치는 로컬, 이름.ckpt or 이름.m5
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, # 가중치만 저장
                             save_best_only=True, # 가장 좋은 결과만 저장
                             monitor='val_loss',  # 기준 = 'validation_loss가 가장 낮은 것'
                             verbose=1) # 출력

In [ ]:
from tensorflow.keras.optimizers import Adam

tuned_Adam = Adam(learning_rate = 0.00035)


model.compile(optimizer = tuned_Adam, loss="categorical_crossentropy", metrics=['ACC'])



In [ ]:
history = model.fit(training_generator,  # IDG 적용 data set
                    validation_data=(validation_generator), # IDG 적용 data set
                    epochs=100, # epoch 230으로 진행 한번 해봐도 될듯
                    callbacks=[checkpoint],
                    )

Epoch 1/100
128/128 [==============================] - ETA: 0s - loss: 0.9374 - ACC: 0.6491 
Epoch 1: val_loss improved from inf to 0.52922, saving model to my_checkpoint1.ckpt
128/128 [==============================] - 1891s 15s/step - loss: 0.9374 - ACC: 0.6491 - val_loss: 0.5292 - val_ACC: 0.8885
Epoch 2/100
128/128 [==============================] - ETA: 0s - loss: 0.4011 - ACC: 0.8894
Epoch 2: val_loss improved from 0.52922 to 0.44648, saving model to my_checkpoint1.ckpt
128/128 [==============================] - 91s 705ms/step - loss: 0.4011 - ACC: 0.8894 - val_loss: 0.4465 - val_ACC: 0.8654
Epoch 3/100
128/128 [==============================] - ETA: 0s - loss: 0.3309 - ACC: 0.9101
Epoch 3: val_loss improved from 0.44648 to 0.40235, saving model to my_checkpoint1.ckpt
128/128 [==============================] - 90s 702ms/step - loss: 0.3309 - ACC: 0.9101 - val_loss: 0.4023 - val_ACC: 0.8885
Epoch 4/100
128/128 [==============================] - ETA: 0s - loss: 0.2671 - ACC: 0.9319

In [ ]:
model.load_weights(checkpoint_path)

test_df = pd.read_csv("/content/drive/MyDrive/mljnu/cnu-mlclass2/"+"/test.csv")
test_df.head(5)
batch_size = len(test_df["image"])
height, width, channel = 224, 224, 3


test_path = "/content/drive/MyDrive/mljnu/cnu-mlclass2/" + '/test'

test_image = np.zeros((batch_size, height, width, channel))
print(test_image.shape)
cnt=0
for i in range(len(test_df["image"])):
    path_img = test_df['image'][i]
    img = tf.keras.preprocessing.image.load_img(os.path.join(test_path,path_img), target_size=(height, width))
    img_tensor = tf.keras.preprocessing.image.img_to_array(img)
    img_tensor = np.array(img_tensor,dtype="float32")

    img_tensor /= 255
    
    img_tensor = np.expand_dims(img_tensor, axis=0)
    
    test_image[i] = img_tensor

predictions = model.predict(test_image)
pred2label={}


for x in training_generator.class_indices.keys():
    pred2label[training_generator.class_indices[x]] = x
test_df['label'] = [pred2label[np.argmax(pred)] for pred in predictions]
test_df
test_df.to_csv('./submission.csv', index=False)

In [ ]:
acc = history.history['ACC']
val_acc = history.history['val_ACC']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = history.epoch

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

print()
print()
print()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()



plt.show()